# Crear el dataset con los precios de los inmuebles por estado

## Vamos a importar las librerias necesarias

In [1]:
import requests
import json
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:,.2f}'.format)

## La función que se mete a la api de centrury 21 y la lista de precios de todos los inmubeles a la venta

In [2]:
def extraer_precio(url):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:138.0) Gecko/20100101 Firefox/138.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'es-ES,es;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Connection': 'keep-alive',
    'Referer': 'https://century21mexico.com/v/resultados/en-pais_mexico/en-estado_ciudad-de-mexico',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'TE': 'trailers'
    }

    r = requests.request("GET", url, headers=headers)
    data = json.loads(r.text)
    data = data['results']
    precios = []
    for inmueble in data:
        precios.append(inmueble['precios']['vista']['precio'])
    return precios

## Función axuiliar para consultar la api y no perder el formato del data set del ISSSTE

In [3]:
def transformar_estados(estados):
    estados_transformados = {}
    for estado in estados:
        estado_transformado = estado.lower().replace(' ', '-').replace('el-', '')
        if 'cdmx' in estado_transformado:
            estado_transformado = 'ciudad-de-mexico'
        estados_transformados[estado] = estado_transformado
    return estados_transformados

## La función principal

In [ ]:
if __name__ == "__main__":
    estados = ['SUR DE LA CDMX', 'ORIENTE DE LA CDMX', 'OAXACA', 'SONORA',
        'EL ESTADO DE MEXICO', 'VERACRUZ', 'GUANAJUATO', 'SAN LUIS POTOSI',
        'HIDALGO', 'CAMPECHE', 'SINALOA', 'TLAXCALA', 'MORELOS',
        'MICHOACAN', 'JALISCO', 'PONIENTE DE LA CDMX', 'TAMAULIPAS',
        'DURANGO', 'GUERRERO', 'BAJA CALIFORNIA SUR', 'PUEBLA',
        'NUEVO LEON', 'CHIHUAHUA', 'NORTE DE LA CDMX', 'NAYARIT',
        'BAJA CALIFORNIA', 'QUINTANA ROO', 'COAHUILA', 'CHIAPAS',
        'ZACATECAS', 'YUCATAN', 'TABASCO', 'QUERETARO', 'AGUASCALIENTES',
        'COLIMA']

    diccionario = transformar_estados(estados)
    resultados = []
    for estado in estados:
        url = f"https://century21mexico.com/v/resultados/operacion_venta/en-pais_mexico/en-estado_{diccionario[estado]}?json=true"
        precios_estado = extraer_precio(url)

        if precios_estado:
            precios_np = np.array(precios_estado, dtype=float)
            q25 = np.percentile(precios_np, 25)
            q50 = np.percentile(precios_np, 50)
            q75 = np.percentile(precios_np, 75)

            resultados.append({
                "estado": estado,
                "q25": q25,
                "q50": q50,
                "q75": q75
            })


    df = pd.DataFrame(resultados)
    df.to_csv("Precios_por_estado.csv", index=False)